In [ ]:
# 寻找外接矩形
import cv2
import numpy as np

# ==========================================================
# 1) 读取图像 —— cv2.imread()
image_np = cv2.imread("../image/31.png")
if image_np is None:
    raise FileNotFoundError("❌ 未找到图片，请检查路径 ../image/31.png")

# 备份一份用于绘制矩形
image_contour = image_np.copy()

# ==========================================================
# 2) 图像灰度化 —— cv2.cvtColor()
# ==========================================================
image_gray = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)

# ==========================================================
# 3) 图像二值化 —— cv2.threshold()
# ==========================================================
# cv2.threshold(src, thresh, maxval, type) -> (ret, dst)
# ----------------------------------------------------------
# 功能：
#   将灰度图像变成黑白图（0 和 255）。
#
# 参数说明：
#   src    : 输入灰度图（必须单通道）
#   thresh : 阈值（0～255）
#   maxval : 设置给“满足条件”的像素值，通常 255
#   type   : 阈值类型，有5大类：
#
#       cv2.THRESH_BINARY
#           像素 > thresh → maxval，否则 0
#
#       cv2.THRESH_BINARY_INV
#           像素 > thresh → 0，否则 maxval（反二值）
#
#       cv2.THRESH_TRUNC
#           像素 > thresh → 设为 thresh
#
#       cv2.THRESH_TOZERO
#           像素 < thresh → 设为 0
#
#       cv2.THRESH_TOZERO_INV
#           像素 > thresh → 设为 0
#
# 返回值：
#   ret : 实际使用的阈值（尤其在 Otsu 自动阈值法中）
#   dst : 二值化后的图像
#
# 注意：
#   - findContours() **只能从白色区域（值=255）中找轮廓**
#   - 所以选择 BINARY 还是 BINARY_INV 取决于前景颜色
# ----------------------------------------------------------
ret, image_thresh = cv2.threshold(image_gray, 40, 255, cv2.THRESH_BINARY_INV)

# ==========================================================
# 4) 轮廓查找 —— cv2.findContours()
# ==========================================================
# cv2.findContours(image, mode, method)
#   -> (contours, hierarchy)
# ----------------------------------------------------------
# 功能：
#   从二值化图像中提取轮廓。
#   ⚠ findContours 认为“白色=前景”，“黑色=背景”。
#
# 参数说明：
#   image  ：二值图（0 或 255）
#   mode   ：轮廓检索模式
#
#       cv2.RETR_EXTERNAL
#           - 仅检测最外层轮廓
#
#       cv2.RETR_LIST
#           - 检测所有轮廓，不建立层级关系
#
#       cv2.RETR_TREE
#           - 检测所有轮廓 + 构建层级结构（父子、兄弟）
#
#   method ：轮廓点近似方法
#
#       cv2.CHAIN_APPROX_NONE
#           - 保存所有边界点（像素级，点非常多）
#
#       cv2.CHAIN_APPROX_SIMPLE（推荐）
#           - 压缩直线段，只保留拐点
#           - 大幅减少点数量，速度更快
#
# 返回值：
#   contours : 列表，每一项是一个轮廓（N×1×2）数组
#   hierarchy: 层级关系（1×轮廓数×4）
#
# hierarchy 的结构：
#   hierarchy[0][i] = [next, prev, child, parent]
#
#     next   : 同级下一个轮廓编号
#     prev   : 同级上一个轮廓编号
#     child  : 子轮廓编号
#     parent : 父轮廓编号
#
# ----------------------------------------------------------
contours, hierarchy = cv2.findContours(
    image_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
)

# ==========================================================
# 5) 绘制轮廓 —— cv2.drawContours()
# ==========================================================
# cv2.drawContours(image, contours, idx, color, thickness)
# ----------------------------------------------------------
# 功能：
#   在图像上绘制轮廓。
#
# 参数说明：
#   image    : 绘制在哪张图像上
#   contours : 轮廓列表
#   idx      : 要绘制哪一个轮廓
#               -1 表示绘制全部轮廓
#   color    : 颜色（B, G, R）
#   thickness: 线宽
# ----------------------------------------------------------
cv2.drawContours(image_contour, contours, -1, (0, 0, 255), 2)

# ==========================================================
# 6) 外接矩形 —— cv2.boundingRect()
# ==========================================================
# cv2.boundingRect(contour)
#   -> (x, y, w, h)
# ----------------------------------------------------------
# 功能：
#   计算轮廓的“水平外接矩形”  
#   （axis-aligned bounding box，不带旋转）
#
# 参数：
#   contour : 某一个轮廓（N×1×2）
#
# 返回值：
#   x, y : 外接矩形左上角坐标
#   w, h : 外接矩形的宽和高
#
# 使用场景：
#   - 目标定位
#   - 图像裁剪 ROI
#   - 目标检测框绘制
#
# 注意：
#   - 如果物体是旋转的矩形，boundingRect 会包含较多空白区域  
#     → 如需旋转外接矩形，请用 cv2.minAreaRect()
# ----------------------------------------------------------
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    top_left = (x, y)
    bottom_right = (x + w, y + h)

    # ======================================================
    # 7) 绘制矩形 —— cv2.rectangle()
    # ======================================================
    # cv2.rectangle(img, pt1, pt2, color, thickness)
    # ------------------------------------------------------
    # 功能：
    #   在图像上绘制矩形框。
    #
    # 参数说明：
    #   img       : 目标图像
    #   pt1       : 左上角坐标 (x1, y1)
    #   pt2       : 右下角坐标 (x2, y2)
    #   color     : BGR 颜色
    #   thickness : 线宽
    #               -1 表示填充矩形
    # ------------------------------------------------------
    cv2.rectangle(image_contour, top_left, bottom_right, (255, 0, 0), 2)

# ==========================================================
# 8) 显示结果
# ==========================================================
cv2.imshow("Threshold Image", image_thresh)
cv2.imshow("Contours + BoundingRect", image_contour)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
# -*- coding: utf-8 -*-
import cv2
import numpy as np

# ==========================================================
# 1) 读取图像 —— cv2.imread()
# ==========================================================
image_np = cv2.imread("../image/31.png")
if image_np is None:
    raise FileNotFoundError("❌ 未找到图片，请检查路径 ../image/31.png")

# 创建一个副本，用于绘制轮廓与外接矩形
image_contour = image_np.copy()


# ==========================================================
# 2) 图像灰度化 —— cv2.cvtColor()
# ==========================================================
image_gray = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)


# ==========================================================
# 3) 二值化 —— cv2.threshold()
# ==========================================================
_, image_thresh = cv2.threshold(image_gray, 40, 255, cv2.THRESH_BINARY_INV)


# ==========================================================
# 4) 轮廓查找 —— cv2.findContours()
# ==========================================================
# cv2.findContours(image, mode, method)
# ----------------------------------------------------------
# 功能：
#   根据二值图像提取轮廓。
#
# 参数：
#   image  : 输入二值图（必须是 0 和 255）
#
#   mode   : 轮廓检索模式
#       cv2.RETR_EXTERNAL —— 只提取最外层轮廓（常用）
#       cv2.RETR_TREE     —— 提取全部轮廓并建立层级结构
#
#   method : 存储轮廓点方式
#       cv2.CHAIN_APPROX_SIMPLE —— 压缩冗余点，只保留拐点
#       cv2.CHAIN_APPROX_NONE   —— 保存所有边界点
#
# 返回值：
#   contours  : Python 列表，每一项都是某个轮廓的所有点
#               contour.shape = (N, 1, 2)
#
#   hierarchy : 轮廓层级关系（父/子/兄弟）
# ----------------------------------------------------------
contours, hierarchy = cv2.findContours(
    image_thresh,
    cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE,  # 在轮廓是直线时，只保留“端点”和“方向改变的拐点”。
)

# ==========================================================
# 5) 绘制所有轮廓 —— cv2.drawContours()
# ==========================================================
# cv2.drawContours(image, contours, contourIdx, color, thickness)
# ----------------------------------------------------------
# 功能：
#   在图像上绘制轮廓边界。
#
# 参数：
#   image      : 绘制到哪张图上
#   contours   : findContours 返回的轮廓列表
#   contourIdx : 要绘制哪个轮廓
#                -1 → 绘制所有轮廓
#   color      : BGR 颜色 (0,0,255)=红色
#   thickness  : 线宽
# ----------------------------------------------------------
cv2.drawContours(image_contour, contours, -1, (0, 0, 255), 2)


# ==========================================================
# 6）最小外接矩形 —— cv2.minAreaRect()
# ==========================================================
# cv2.minAreaRect(contour)
# ----------------------------------------------------------
# 功能：
#   寻找轮廓的 *最小面积旋转矩形*（即允许倾斜）。
#
# 参数：
#   contour : 某一个轮廓（N×1×2 的数组）
#
# 返回：
#   rect = (center, size, angle)
#
#   rect[0] center : (cx, cy) 旋转矩形中心坐标（浮点数）
#   rect[1] size   : (w, h)   旋转矩形的宽和高（浮点数）
#   rect[2] angle  : 旋转角度（单位：度）
#
# 注意：
#   minAreaRect 返回的是数学矩形，不提供四个点坐标，
#   所以必须用 boxPoints 才能取得矩形四个顶点。
# ----------------------------------------------------------

# ==========================================================
# 7）矩形四点计算 —— cv2.boxPoints()
# ==========================================================
# cv2.boxPoints(rect) → 4×2 的顶点坐标数组
# ----------------------------------------------------------
# 功能：
#   将 minAreaRect 返回的旋转矩形（中心+宽高+角度）
#   转换四个顶点的坐标
#
# 返回：
#   4 个坐标点，顺序是：
#       左上 → 右上 → 右下 → 左下（顺时针）
#
# 注意：
#   返回结果是浮点，需要转 int 才能绘制。
# ----------------------------------------------------------

for cnt in contours:
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    box = np.int64(box)  # 转为整数索引

    # ======================================================
    # 8）绘制旋转外接矩形 —— drawContours()
    # ======================================================
    # drawContours() 第2个参数必须是“轮廓列表”，
    # 所以需要用 [box] 包装成列表。
    # ------------------------------------------------------
    # drawContours 不关心你传入的是“真实轮廓”还是“任意点集”，它只画点集之间的连线。
    cv2.drawContours(
        image_contour,
        [box],  # list 类型
        -1,  # 绘制此轮廓
        (255, 0, 0),  # 蓝色
        2,
    )


# ==========================================================
# 9) 显示结果 —— cv2.imshow()
# ==========================================================
cv2.imshow("Threshold Image", image_thresh)
cv2.imshow("Contours + MinAreaRect", image_contour)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# 寻找最小外接圆
import cv2
import numpy as np

# ==========================================================
# 1) 读取图像 —— cv2.imread()
image_np = cv2.imread("../image/31.png")
if image_np is None:
    raise FileNotFoundError("❌ 未找到图片，请检查路径 ../image/31.png")

# 备份一份用于绘制矩形
image_contour = image_np.copy()

# ==========================================================
# 2) 图像灰度化 —— cv2.cvtColor()
# ==========================================================
image_gray = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)

# ==========================================================
# 3) 图像二值化 —— cv2.threshold()
ret, image_thresh = cv2.threshold(image_gray, 40, 255, cv2.THRESH_BINARY_INV)

contours, hierarchy = cv2.findContours(
    image_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
)

cv2.drawContours(image_contour, contours, -1, (0, 0, 255), 2)


for cnt in contours:

    # ==========================================================
    # 1）最小外接圆 —— cv2.minEnclosingCircle()
    # ==========================================================
    # cv2.minEnclosingCircle(points) -> (center, radius)
    # ----------------------------------------------------------
    # 功能：
    #   计算包围轮廓（点集）面积最小的圆（最小外接圆）。
    #
    # 参数：
    #   points : 某一个轮廓（N×1×2 数组），即 findContours 返回的每个 cnt。
    #
    # 返回值：
    #   (x, y):
    #       圆心坐标（浮点数）。注意：minEnclosingCircle 的圆心可能不是整数，
    #       因为它根据几何优化计算而来，不是像素对齐。
    #
    #   radius:
    #       圆的半径（浮点数）。
    #
    # 注意：
    #   - minEnclosingCircle 不要求轮廓是凸的，它会自动优化找到能包住整个轮廓的最小圆。
    #   - 返回值均为浮点，需要转换成 int 才能用于绘图。
    # ----------------------------------------------------------
    (x, y), radius = cv2.minEnclosingCircle(cnt)

    # 将浮点转为整数，便于画圆
    (x, y, radius) = np.int64((x, y, radius))

    # ==========================================================
    # 2）绘制最小外接圆 —— cv2.circle()
    # ==========================================================
    # cv2.circle(img, center, radius, color, thickness)
    # ----------------------------------------------------------
    # 功能：
    #   在图像上绘制圆形（实心或空心）。
    #
    # 参数：
    #   img       : 要绘制的目标图像
    #   center    : 圆心坐标 (x, y)，必须是整数
    #   radius    : 圆的半径
    #
    #   color     : BGR 颜色，例如：
    #                   (255, 0, 0) = 蓝色
    #
    #   thickness : 线宽
    #               - 2 表示线宽为 2 像素
    #               - -1 表示填充整个圆
    #
    # 说明：
    #   - 使用 minEnclosingCircle + cv2.circle 常用于形状包装、目标定位、
    #     粒子检测、医学图像处理，以及圆形目标识别。
    # ----------------------------------------------------------
    cv2.circle(
        image_contour,
        (x, y),  # 圆心坐标
        radius,  # 半径
        (255, 0, 0),  # 蓝色
        2,  # 线宽
    )


cv2.imshow("Threshold Image", image_thresh)
cv2.imshow("Contours + BoundingRect", image_contour)
cv2.waitKey(0)
cv2.destroyAllWindows()